# Pandas

We'll breeze through the basics here, and get onto some assignments in a bit. I want to provide the *barest* of intuition so things stick down the road.

## Why pandas?

NumPy is great. But it lacks a few things that are conducive to doing statisitcal analysis. By building on top of NumPy, pandas provides

- labeled arrays
- heterogenous data types within a table
- "better" missing data handling
- convenient methods (`groupby`, `rolling`, `resample`)
- more data types (Categorical, Datetime)

## Data Structures

This is the typical starting point for any intro to pandas.
We'll follow suit.

### The DataFrame

Here we have the workhorse data structure for pandas.
It's an in-memory table holding your data, and provides a few conviniences over lists of lists or NumPy arrays.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Many ways to construct a DataFrame
# We pass a dict of {column name: column values}
np.random.seed(42)
df = pd.DataFrame({'A': [1, 2, 3], 
                   'B': [True, True, False],
                   'C': np.random.randn(3)},
                  index=['a', 'b', 'c'])  # also this weird index thing
df

,A,B,C
a,1,True,0.496714
b,2,True,-0.138264
c,3,False,0.647689


Notice that we can store a column of intergers, a column of booleans, and a column of floats in the same `DataFrame`.

### Indexing

Our first improvement over numpy arrays is labeled indexing. We can select subsets by column, row, or both. Column selection uses the regular python `__getitem__` machinery. Pass in a single column label `'A'` or a list of labels `['A', 'C']` to select subsets of the original `DataFrame`.

In [3]:
# Single column, reduces to a Series
df['A']

a    1
b    2
c    3
Name: A, dtype: int64

In [4]:
cols = ['A', 'C']
df[cols]

,A,C
a,1,0.496714
b,2,-0.138264
c,3,0.647689


For row-wise selection, use the special `.loc` accessor.

In [5]:
df.loc[['a', 'b']]

,A,B,C
a,1,True,0.496714
b,2,True,-0.138264


You can use ranges to select rows or columns.

In [6]:
df.loc['a':'b']

,A,B,C
a,1,True,0.496714
b,2,True,-0.138264


Notice that the slice is *inclusive* on both sides,  unlike your typical slicing of a list. Sometimes, you'd rather slice by *position* instead of label. `.iloc` has you covered:

In [7]:
df.iloc[[0, 1]]

,A,B,C
a,1,True,0.496714
b,2,True,-0.138264


In [8]:
df.iloc[:2]

,A,B,C
a,1,True,0.496714
b,2,True,-0.138264


This follows the usual python slicing rules: closed on the left, open on the right.

As I mentioned, you can slice both rows and columns. Use `.loc` for label or `.iloc` for position indexing.

In [9]:
df.loc['a', 'B']

True

Pandas, like NumPy, will reduce dimensions when possible. Select a single column and you get back `Series` (see below). Select a single row and single column, you get a scalar.

You can get pretty fancy:

In [10]:
df.loc['a':'b', ['A', 'C']]

,A,C
a,1,0.496714
b,2,-0.138264


#### Summary

- Use `[]` for selecting columns
- Use `.loc[row_lables, column_labels]` for label-based indexing
- Use `.iloc[row_positions, column_positions]` for positional index

I've left out boolean and hierarchical indexing, which we'll see later.

## Series

You've already seen some `Series` up above. It's the 1-dimensional analog of the DataFrame. Each column in a `DataFrame` is in some sense a `Series`. You can select a `Series` from a DataFrame in a few ways:

In [11]:
# __getitem__ like before
df['A']

a    1
b    2
c    3
Name: A, dtype: int64

In [12]:
# .loc, like before
df.loc[:, 'A']

a    1
b    2
c    3
Name: A, dtype: int64

In [13]:
# using `.` attribute lookup
df.A

a    1
b    2
c    3
Name: A, dtype: int64

In [ ]:
df['mean'] = ['a', 'b', 'c']

In [ ]:
df['mean']

In [ ]:
df.mean

You'll have to be careful with the last one. It won't work if you're column name isn't a valid python identifier (say it has a space) or if it conflicts with one of the (many) methods on `DataFrame`. The `.` accessor is extremely convient for interactive use though.

You should never *assign* a column with `.` e.g. don't do

```python
# bad
df.A = [1, 2, 3]
```

It's unclear whether your attaching the list `[1, 2, 3]` as an attribute of `df`, or whether you want it as a column. It's better to just say

```python
df['A'] = [1, 2, 3]
# or
df.loc[:, 'A'] = [1, 2, 3]
```

`Series` share many of the same methods as `DataFrame`s.

## Index

`Index`es are something of a peculiarity to pandas.
First off, they are not the kind of indexes you'll find in SQL, which are used to help the engine speed up certain queries.
In pandas, `Index`es are about lables. This helps with selection (like we did above) and automatic alignment when performing operations between two `DataFrame`s or `Series`.

R does have row labels, but they're nowhere near as powerful (or complicated) as in pandas. You can access the index of a `DataFrame` or `Series` with the `.index` attribute.

In [14]:
df.index

Index(['a', 'b', 'c'], dtype='object')

In [15]:
df.columns

Index(['A', 'B', 'C'], dtype='object')

## Operations

In [16]:
np.random.seed(42)
df = pd.DataFrame(np.random.uniform(0, 10, size=(3, 3)))
df

,0,1,2
0,3.745401,9.507143,7.319939
1,5.986585,1.560186,1.559945
2,0.580836,8.661761,6.011150


In [17]:
df + 1

,0,1,2
0,4.745401,10.507143,8.319939
1,6.986585,2.560186,2.559945
2,1.580836,9.661761,7.011150


In [18]:
df ** 2

,0,1,2
0,14.028030,90.385769,53.581513
1,35.839198,2.434182,2.433429
2,0.337371,75.026112,36.133926


In [19]:
np.log(df)

,0,1,2
0,1.320529,2.252043,1.990602
1,1.789521,0.444805,0.444651
2,-0.543287,2.158918,1.793616


DataFrames and Series have a bunch of useful aggregation methods, `.mean`, `.max`, `.std`, etc.

In [20]:
df.mean()

0    3.437607
1    6.576364
2    4.963678
dtype: float64

## Loading Data

In [22]:
df = pd.read_csv('beer_subset.csv.gz', parse_dates=['time'], compression='gzip')
review_cols = ['review_appearance', 'review_aroma', 'review_overall',
               'review_palate', 'review_taste']
df.head(100)

,abv,beer_id,brewer_id,beer_name,beer_style,review_appearance,review_aroma,review_overall,review_palate,profile_name,review_taste,text,time
0,7.0,2511,287,Bell's Cherry Stout,American Stout,4.5,4.0,4.5,4.0,blaheath,4.5,Batch 8144\tPitch black in color with a 1/2 f...,2009-10-05 21:31:48
1,5.7,19736,9790,Duck-Rabbit Porter,American Porter,4.5,4.0,4.5,4.0,GJ40,4.0,Sampled from a 12oz bottle in a standard pint...,2009-10-05 21:32:09
2,4.8,11098,3182,Fürstenberg Premium Pilsener,German Pilsener,4.0,3.0,3.0,3.0,biegaman,3.5,Haystack yellow with an energetic group of bu...,2009-10-05 21:32:13
3,9.5,28577,3818,Unearthly (Imperial India Pale Ale),American Double / Imperial IPA,4.0,4.0,4.0,4.0,nick76,4.0,"The aroma has pine, wood, citrus, caramel, an...",2009-10-05 21:32:37
4,5.8,398,119,Wolaver's Pale Ale,American Pale Ale (APA),4.0,3.0,4.0,3.5,champ103,3.0,A: Pours a slightly hazy golden/orange color....,2009-10-05 21:33:14
5,7.0,966,365,Pike Street XXXXX Stout,American Stout,4.0,4.0,3.5,4.0,sprucetip,4.5,"From notes. Pours black, thin mocha head fade...",2009-10-05 21:33:48
6,6.2,53128,1114,Smokin' Amber Kegs Gone Wild,American Amber / Red Ale,3.5,4.0,4.5,4.0,Deuane,4.5,An American amber with the addition of smoked...,2009-10-05 21:34:24
7,4.8,1669,256,Great White,Witbier,4.5,4.5,4.5,4.5,n0rc41,4.5,"Ok, for starters great white I believe will b...",2009-10-05 21:34:29
8,6.7,6549,140,Northern Hemisphere Harvest Wet Hop Ale,American IPA,4.0,4.0,4.0,4.0,david18,4.0,I like all of Sierra Nevada's beers but felt ...,2009-10-05 21:34:31
9,6.5,13824,743,Oktoberfest,Vienna Lager,3.0,2.5,2.5,2.5,Seanibus,2.5,This actually winds up coming out like a ligh...,2009-10-05 21:35:09


## Boolean indexing

Like a where clause in SQL. The indexer (or boolean mask) should be 1-dimensional and the same length as the thing being indexed.

In [23]:
df.abv < 5

0      False
1      False
2       True
3      False
4      False
5      False
6      False
7       True
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21      True
22      True
23     False
24     False
25     False
26     False
27     False
28      True
29     False
       ...  
969    False
970    False
971    False
972    False
973    False
974    False
975    False
976    False
977    False
978    False
979    False
980    False
981    False
982    False
983    False
984    False
985    False
986    False
987    False
988    False
989    False
990    False
991    False
992    False
993    False
994    False
995    False
996    False
997    False
998    False
Name: abv, Length: 999, dtype: bool

In [24]:
df[df.abv < 5]

,abv,beer_id,brewer_id,beer_name,beer_style,review_appearance,review_aroma,review_overall,review_palate,profile_name,review_taste,text,time
2,4.80,11098,3182,Fürstenberg Premium Pilsener,German Pilsener,4.0,3.0,3.0,3.0,biegaman,3.5,Haystack yellow with an energetic group of bu...,2009-10-05 21:32:13
7,4.80,1669,256,Great White,Witbier,4.5,4.5,4.5,4.5,n0rc41,4.5,"Ok, for starters great white I believe will b...",2009-10-05 21:34:29
21,4.60,401,118,Dark Island,Scottish Ale,4.0,4.0,3.5,4.0,abuliarose,4.0,"Poured into a snifter, revealing black opaque...",2009-10-05 21:47:36
22,4.90,5044,18968,Kipona Fest,Märzen / Oktoberfest,4.0,3.5,4.0,4.0,drcarver,4.0,A - a medium brown body with an off white hea...,2009-10-05 21:47:56
28,4.60,401,118,Dark Island,Scottish Ale,4.0,4.0,4.5,4.0,sisuspeed,4.0,The color of this beer fits the name well. Op...,2009-10-05 21:53:38
30,4.90,909,306,Killian's Irish Red,American Amber / Red Lager,4.0,3.5,3.5,3.0,tone77,3.0,Poured from a brown 12oz. bottle. Has an ambe...,2009-10-05 22:01:09
40,4.10,51464,29,Bud Light Golden Wheat,Herbed / Spiced Beer,3.0,3.0,3.5,3.5,armock,3.5,A - Poured a hazy orange color with a white h...,2009-10-05 22:11:53
75,4.00,53136,6045,English Session Ale,English Bitter,4.0,3.5,4.0,3.5,beerthulhu,3.5,4.0% abv. \t\tA: Poured into a pint with a ro...,2009-10-05 22:55:10
77,4.10,9166,6045,Raspberry Wheat,Fruit / Vegetable Beer,3.0,3.5,3.5,3.5,beerthulhu,4.0,"A: Poured a crystal golen-orange with a thin,...",2009-10-05 22:57:38
80,4.50,2970,91,Taj Mahal Premium Lager,American Adjunct Lager,3.0,2.5,2.5,2.5,wertperch,2.5,I had one of these at a local curry house. Po...,2009-10-05 23:06:44


Notice that we just used `[]` there. We can pass the boolean indexer in to `.loc` as well.

In [ ]:
df.loc[df.abv < 5, ['beer_style', 'review_overall']].head()

Again, you can get complicated

In [25]:
df[((df.abv < 5) & (df.time > pd.Timestamp('2009-06'))) | (df.review_overall >= 4.5)]

,abv,beer_id,brewer_id,beer_name,beer_style,review_appearance,review_aroma,review_overall,review_palate,profile_name,review_taste,text,time
0,7.00,2511,287,Bell's Cherry Stout,American Stout,4.5,4.0,4.5,4.0,blaheath,4.5,Batch 8144\tPitch black in color with a 1/2 f...,2009-10-05 21:31:48
1,5.70,19736,9790,Duck-Rabbit Porter,American Porter,4.5,4.0,4.5,4.0,GJ40,4.0,Sampled from a 12oz bottle in a standard pint...,2009-10-05 21:32:09
2,4.80,11098,3182,Fürstenberg Premium Pilsener,German Pilsener,4.0,3.0,3.0,3.0,biegaman,3.5,Haystack yellow with an energetic group of bu...,2009-10-05 21:32:13
6,6.20,53128,1114,Smokin' Amber Kegs Gone Wild,American Amber / Red Ale,3.5,4.0,4.5,4.0,Deuane,4.5,An American amber with the addition of smoked...,2009-10-05 21:34:24
7,4.80,1669,256,Great White,Witbier,4.5,4.5,4.5,4.5,n0rc41,4.5,"Ok, for starters great white I believe will b...",2009-10-05 21:34:29
10,11.80,43670,423,Bourbon Barrel Quad (BBQ),Quadrupel (Quad),4.5,4.0,4.5,4.5,blaheath,4.5,"Burnt amber in color with a 1/4"" head. Aroma ...",2009-10-05 21:36:03
11,6.30,53129,14365,Seneca Pale Ale,American Pale Ale (APA),3.0,3.5,4.5,4.0,Daniellobo,4.0,A very pleasing PA leaning strongly on the IP...,2009-10-05 21:37:48
13,8.00,652,24,Traquair Jacobite,Scotch Ale / Wee Heavy,4.5,4.5,4.5,4.5,drtth,4.5,Lightly chilled and poured into a Duvel glass...,2009-10-05 21:40:18
21,4.60,401,118,Dark Island,Scottish Ale,4.0,4.0,3.5,4.0,abuliarose,4.0,"Poured into a snifter, revealing black opaque...",2009-10-05 21:47:36
22,4.90,5044,18968,Kipona Fest,Märzen / Oktoberfest,4.0,3.5,4.0,4.0,drcarver,4.0,A - a medium brown body with an off white hea...,2009-10-05 21:47:56


<div class="alert alert-success">
    <b>Exercise</b>: Find the American beers
</div>

Select just the rows where the `beer_style` contains `'American'`. 

Hint: `Series` containing strings have a bunch of [useful methods](http://pandas.pydata.org/pandas-docs/stable/text.html#method-summary) under the `DataFrame.<column>.str` namespace. Typically they correspond to regular python string methods, but

- They gracefully propogate missing values
- They're a bit more liberal about accepting regular expressions

We can't use `'American' in df['beer_style']`, since `in` is used to check membership in the series itself, not the strings. But `in` uses `__contains__`, so look for a string method like that.

In [ ]:
df.beer_style.str.contains("American")

In [ ]:
# Your solution
is_ipa = df.beer_style.str.contains("American")
df[is_ipa]

## Groupby

Groupby is a fundamental operation to pandas and data analysis.

The components of a groupby operation are to

1. Split a table into groups
2. Apply a function to each group
3. Combine the results

In pandas the first step looks like

```python
df.groupby( grouper )
```

`grouper` can be many things

- Series (or string indicating a column in `df`)
- function (to be applied on the index)
- dict : groups by *values*
- `levels=[ names of levels in a MultiIndex ]`

In [26]:
gr = df.groupby('beer_style')
gr

Haven't really done anything yet. Just some book-keeping to figure out which **keys** go with which **rows**. Keys are the things we've grouped by (each `beer_style` in this case).

The last two steps, apply and combine, are just:

In [27]:
gr.agg('mean')

,abv,beer_id,brewer_id,review_appearance,review_aroma,review_overall,review_palate,review_taste
beer_style,,,,,,,,
Altbier,5.850000,43260.500000,419.500000,4.000000,3.750000,4.000000,3.750000,4.000000
American Adjunct Lager,4.872727,12829.909091,2585.909091,2.954545,2.613636,3.272727,2.909091,2.750000
American Amber / Red Ale,6.195652,28366.777778,2531.111111,3.740741,3.592593,3.870370,3.555556,3.777778
American Amber / Red Lager,4.822857,22277.500000,5620.125000,3.437500,3.312500,3.375000,3.187500,3.125000
American Barleywine,10.208333,32457.250000,3744.083333,3.958333,3.937500,3.729167,3.895833,3.937500
American Black Ale,8.600000,48415.500000,10051.500000,4.500000,4.250000,3.750000,4.000000,4.000000
American Blonde Ale,5.488889,22984.800000,2672.800000,3.200000,2.850000,2.900000,2.900000,3.000000
American Brown Ale,6.021875,20128.812500,5674.687500,3.812500,3.968750,3.937500,3.687500,3.906250
American Double / Imperial IPA,9.163425,32345.219178,3857.753425,4.178082,4.164384,3.986301,4.095890,4.136986


In [ ]:
df.groupby('beer_style').mean()

This says apply the `mean` function to each column. Non-numeric columns (nusiance columns) are excluded. We can also select a subset of columns to perform the aggregation on.

In [ ]:
gr[review_cols].agg('mean')

`.` attribute lookup works as well.

In [ ]:
gr.abv.agg('mean')

Certain operations are attached directly to the `GroupBy` object, letting you bypass the `.agg` part

In [ ]:
gr.abv.mean()

Now we'll run the gamut on a bunch of grouper / apply combinations.
Keep sight of the target though: split, apply, combine.

- Grouper: Controls the output index
    * single grouper -> `Index`
    * array-like grouper -> `MultiIndex`
- Subject (Groupee): Controls the output data values
    * single column -> `Series` (or DataFrame if multiple aggregations)
    * multiple columns -> `DataFrame`
- Aggregation: Controls the output columns
    * single aggfunc -> `Index` in the colums
    * multiple aggfuncs -> `MultiIndex` in the columns (Or 1-D Index groupee is 1-d)

Multiple Aggregations on one column

In [28]:
gr['review_aroma'].agg(['mean', 'std', 'count']).head()

,mean,std,count
beer_style,,,
Altbier,3.750000,0.353553,2
American Adjunct Lager,2.613636,0.596255,22
American Amber / Red Ale,3.592593,0.636049,27
American Amber / Red Lager,3.312500,0.842509,8
American Barleywine,3.937500,0.449940,24


Single Aggregation on multiple columns

In [ ]:
gr[review_cols].mean()

Multiple aggregations on multiple columns

In [ ]:
gr[review_cols].agg(['mean', 'count', 'std'])

Hierarchical Indexes in the columns can be awkward to work with, so I'll usually
move a level to the Index with `.stack`.

In [ ]:
multi = gr[review_cols].agg(['mean', 'count', 'std']).stack(level=0)
multi.head(10)

You can group by **levels** of a MultiIndex.

In [ ]:
multi.groupby(level='beer_style')['mean'].agg(['min', 'max'])

Group by **multiple** columns

In [ ]:
df.groupby(['brewer_id', 'beer_style']).review_overall.mean()

In [ ]:
df.groupby(['brewer_id', 'beer_style'])[review_cols].mean()